# Using the EBAY API 

## Finding API 2022


<img align="left" width="100" height="100" src="ebay.png">

## Install EBAY SDK and required packages

#!pip3 install ebaysdk
#!pip3 install python-dotenv

import pandas as pd
from ebaysdk.finding import Connection as Finding
from ebaysdk.exception import ConnectionError
import os

In [82]:
from dotenv import load_dotenv
load_dotenv()

True

In [83]:
print(os.getcwd())
!python3 --version

/home/rag
Python 3.8.10


In [84]:
# Deprecation
# pd.set_option('max_colwidth', 1000)
# pd.set_option('max_rows', 1000)
pd.set_option('display.max_rows', 500)

## Hide your Application ID in a .env file

In [85]:
APPLICATION_ID = os.environ.get('API_KEY')

## Function to get the payload

In [86]:
def get_results(payload):
    try:
        api = Finding(siteid='EBAY-GB', appid=APPLICATION_ID, config_file=None)
        response = api.execute('findItemsAdvanced', payload)
        return response.dict()
    except ConnectionError as e:
        print(e)
        print(e.response.dict())

### Define the payload

In [87]:
payload = {
        'keywords': '12 string bass guitar', 
        'categoryId': ['3858'],
        'itemFilter': [
            {'name': 'LocatedIn', 'value': 'GB'},
        ],
        'sortOrder': 'StartTimeNewest',
}

In [88]:
results = get_results(payload)

In [89]:
def get_total_pages(results):
    '''Get the total number of pages from the results'''
    if results:
        return int(results.get('paginationOutput').get('totalPages'))
    else:
        return

In [90]:
def search_ebay(payload):
    '''parse the response - results and concatentate to the dataframe'''
    results = get_results(payload)
    total_pages = get_total_pages(results)
    items_list = results['searchResult']['item']
        
    i = 2
    while(i <= total_pages):
        payload['paginationInput'] = {'entriesPerPage': 100, 'pageNumber': i}        
        results = get_results(payload)
        items_list.extend(results['searchResult']['item'])
        i += 1
        
    df_items = pd.DataFrame(columns=['itemId', 'title', 'viewItemURL', 'galleryURL', 'location', 'postalCode',
                                 'paymentMethod''listingType', 'bestOfferEnabled', 'buyItNowAvailable',
                                 'currentPrice', 'bidCount', 'sellingState'])

    for item in items_list:
        row = {
            'itemId': item.get('itemId'),
            'title': item.get('title'),
            'viewItemURL': item.get('viewItemURL'),
            'galleryURL': item.get('galleryURL'),
            'location': item.get('location'),
            'postalCode': item.get('postalCode'),
            'paymentMethod': item.get('paymentMethod'),        
            'listingType': item.get('listingInfo').get('listingType'),
            'bestOfferEnabled': item.get('listingInfo').get('bestOfferEnabled'),
            'buyItNowAvailable': item.get('listingInfo').get('buyItNowAvailable'),
            'currentPrice': item.get('sellingStatus').get('currentPrice').get('value'),
            'bidCount': item.get('bidCount'),
            'sellingState': item.get('sellingState'),
        }

        ## Deprecation note - don't use append
        '''https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html'''
        
        new_df = pd.DataFrame([row])
        df_items = pd.concat([df_items, new_df],axis=0, ignore_index=True)


    return df_items


# Run the search

In [91]:
df = search_ebay(payload)

In [92]:
df

,itemId,title,viewItemURL,galleryURL,location,postalCode,paymentMethodlistingType,bestOfferEnabled,buyItNowAvailable,currentPrice,bidCount,sellingState,paymentMethod,listingType
0,353937596446,jackson surfcaster 12 string guitar sea foam g...,https://www.ebay.co.uk/itm/jackson-surfcaster-...,https://i.ebayimg.com/thumbs/images/g/vaUAAOSw...,"Fareham,United Kingdom",PO160HZ,NaN,true,false,1950.0,None,None,None,FixedPrice
1,154864091298,Kimbara/crafta /Yamaha guitars 12 and 6 string...,https://www.ebay.co.uk/itm/Kimbara-crafta-Yama...,https://i.ebayimg.com/thumbs/images/g/vuEAAOSw...,"London,United Kingdom",N99**,NaN,true,false,240.0,None,None,None,Auction
2,165345176091,Cort 12 String,https://www.ebay.co.uk/itm/Cort-12-String-/165...,https://i.ebayimg.com/thumbs/images/g/P2cAAOSw...,"Stowmarket,United Kingdom",IP145RX,NaN,true,false,120.0,None,None,CashOnPickup,FixedPrice
3,144384855149,Veillette-Citron Guitar vintage advert 1978 el...,https://www.ebay.co.uk/itm/Veillette-Citron-Gu...,https://i.ebayimg.com/thumbs/images/g/Ms8AAOSw...,"Bognor Regis,United Kingdom",PO213QN,NaN,true,false,5.95,None,None,CashOnPickup,FixedPrice
4,234358274961,Glarry GIB Electric Bass Guitar Full Size 4 St...,https://www.ebay.co.uk/itm/Glarry-GIB-Electric...,https://i.ebayimg.com/thumbs/images/g/9p8AAOSw...,"Milton Keynes,United Kingdom",MK94AT,NaN,false,false,131.99,None,None,None,FixedPrice
5,234358267603,Glarry GIB Electric 5 String Bass Guitar Full ...,https://www.ebay.co.uk/itm/Glarry-GIB-Electric...,https://i.ebayimg.com/thumbs/images/g/GBkAAOSw...,"Milton Keynes,United Kingdom",MK94AT,NaN,false,false,97.99,None,None,None,FixedPrice
6,304239582417,Vintage Brand New GHS Dulcimer Loopend Stainle...,https://www.ebay.co.uk/itm/Vintage-Brand-New-G...,https://i.ebayimg.com/thumbs/images/g/rdoAAOSw...,"Nelson,United Kingdom",BB98NR,NaN,false,false,15.0,None,None,None,FixedPrice
7,255244423945,50 Acoustic Guitar Bridge Pins White Cream Bla...,https://www.ebay.co.uk/itm/50-Acoustic-Guitar-...,https://i.ebayimg.com/thumbs/images/g/wlcAAOSw...,"Brighton,United Kingdom",BN21JH,NaN,false,false,4.74,None,None,None,FixedPrice
8,255239963910,50 Acoustic Guitar Bridge Pins White Cream Bla...,https://www.ebay.co.uk/itm/50-Acoustic-Guitar-...,https://i.ebayimg.com/thumbs/images/g/bMAAAOSw...,"Brighton,United Kingdom",BN21JH,NaN,false,false,4.99,None,None,None,FixedPrice
9,255232438064,12 X Acoustic Guitar Bridge Pins White Cream B...,https://www.ebay.co.uk/itm/12-X-Acoustic-Guita...,https://i.ebayimg.com/thumbs/images/g/3-sAAOSw...,"Brighton,United Kingdom",BN21JH,NaN,true,false,4.99,None,None,None,FixedPrice
